# 函数定义详解

## 1. 默认值参数

In [1]:
def ask_ok(prompt, retries=4, reminder="Please try again!"):
    while True:
        reply = input(prompt)
        if reply in {"y", "ye", "yes"}:
            return True
        if reply in {"n", "no", "nop", "nope"}:
            return False
        retries = retries -1
        if retries < 0:
            raise ValueError("invalid user response")
        print(reminder)

为参数指定默认值,可以在调用函数时,使用比函数定义更少的参数:

- 只给出必选实参

In [2]:
ask_ok('Do you really want to quit?')

Do you really want to quit? ewq


Please try again!


Do you really want to quit? ewq


Please try again!


Do you really want to quit? eq


Please try again!


Do you really want to quit? eq


Please try again!


Do you really want to quit? eq


ValueError: invalid user response

- 只给出一个可选实参

In [4]:
ask_ok('Do you really want to quit?', 2)

Do you really want to quit? 1


Please try again!


Do you really want to quit? 1


Please try again!


Do you really want to quit? 1


ValueError: invalid user response

- 给出所有实参

In [5]:
ask_ok('OK to overwrite the file?', 2, 'Come on, only yes or no!')

OK to overwrite the file? 1


Come on, only yes or no!


OK to overwrite the file? 1


Come on, only yes or no!


OK to overwrite the file? 1


ValueError: invalid user response

- **重要警告**

- 默认参数在函数定义时求值,而不是调用时,默认值只计算一次,后续修改不影响默认值

In [7]:
i = 5

def f(arg=i):  # 等价于 arg=5
    print(arg)

i = 6  # 此处修改不影响函数的默认参数
f()

5


- 对于可变对象可能导致意外的行为

In [9]:
# 列表默认参数的陷阱

def f(a, L=[]):
    L.append(a)
    return L

In [10]:
print(f(1))
print(f(2))  # 使用的是同一个列表

[1]
[1, 2]


In [12]:
# 避免参数的正确写法

def f(a, L=None):
    if L is None:
        L = []
    L.append(a)
    return L        

In [13]:
print(f(1))
print(f(2))

[1]
[2]


## 2. 关键字擦参数

关键字参数采用 kwarg=value 形式,函数示例如下:

In [14]:
def parrot(voltage, state='a stiff', action='voom', type='Norwegian Blue'):
    print("-- This parrot would't", action, end=' ')
    print("if you put", voltage, "voltage through it")
    print("-- Lovely plumage, the", type)
    print("-- It's", state, "!")

该函数接受一个必选参数（voltage）和三个可选参数（state, action 和 type）。该函数可用下列方式调用：

In [15]:
parrot(1000)  # 1个位置参数

-- This parrot would't voom if you put 1000 voltage through it
-- Lovely plumage, the Norwegian Blue
-- It's a stiff !


In [16]:
parrot(voltage=1000)  # 1个关键参数

-- This parrot would't voom if you put 1000 voltage through it
-- Lovely plumage, the Norwegian Blue
-- It's a stiff !


In [17]:
parrot(voltage=1000000, action="V00000M")  # 2个关键字参数

-- This parrot would't V00000M if you put 1000000 voltage through it
-- Lovely plumage, the Norwegian Blue
-- It's a stiff !


In [18]:
parrot(action='VOOOOOM', voltage=1000000)  # 2个关键字参数

-- This parrot would't VOOOOOM if you put 1000000 voltage through it
-- Lovely plumage, the Norwegian Blue
-- It's a stiff !


In [19]:
parrot('a million', 'bereft of life', 'jump')  # 3个关键字参数

-- This parrot would't jump if you put a million voltage through it
-- Lovely plumage, the Norwegian Blue
-- It's bereft of life !


In [20]:
parrot('a thousand', state='pushing up the daisies')  # 1 个位置参数，1 个关键字参数

-- This parrot would't voom if you put a thousand voltage through it
-- Lovely plumage, the Norwegian Blue
-- It's pushing up the daisies !


函数调用时,关键字参数必须跟在位置参数后面.所有传递的关键字参数都必须匹配一个函数接受的参数,关键字参数的顺序并不重要。

最后一个形参为 ****name** 形式时,接收一个字典,该字典包含与函数中已定义形参对应之外的所有关键字参数。

In [21]:
def cheeseshop(kind, **keywords):
    print("-- Do you have any", kind, "?")
    print("-- I'm sorry, we are all out of", kind)
    for kw in keywords:
        print(kw, ":", keywords[kw])

In [22]:
cheeseshop("Limburger", 
           shopkeeper="Michael Palin",
           client="John Cleese",
           sketch="Cheese Shop Sketch"
          )

-- Do you have any Limburger ?
-- I'm sorry, we are all out of Limburger
shopkeeper : Michael Palin
client : John Cleese
sketch : Cheese Shop Sketch


***name** 形参接收一个元组,该元组包含形参列表之外的位置参数.

In [23]:
def cheeseshop(kind, *arguments):
    print("-- Do you have any", kind, "?")
    print("-- I'm sorry, we are all out of", kind)
    for arg in arguments:
        print(arg)

In [24]:
cheeseshop("Limburger", 
           "It's very runny, sir.",
            "It's really very, VERY runny, sir."
          )

-- Do you have any Limburger ?
-- I'm sorry, we are all out of Limburger
It's very runny, sir.
It's really very, VERY runny, sir.


## 3. 特殊参数

默认情况下，参数可以按位置或显式关键字传递给 Python 函数。为了让代码易读、高效，最好限制参数的传递方式，这样，开发者只需查看函数定义，即可确定参数项是仅按位置、按位置或关键字，还是仅按关键字传递。函数定义如下:

def f(pos1, pos2, /, pos_or_kwd, *, kwd1, kwd2):
      -----------    ----------     ----------
        |             |                  |
        |        位置或关键字   |
        |                                - 仅限关键字
         -- 仅限位置

/ 和 * 是可选的。这些符号表明形参如何把参数值传递给函数：位置、位置或关键字、关键字。关键字形参也叫作命名形参。

### 3.1 位置或关键字参数

函数定义中未使用 / 和 * 时，参数可以按位置或关键字传递给函数。

In [25]:
def standard_arg(arg):
    print(arg)

In [27]:
standard_arg(1)

TypeError: standard_arg() takes 1 positional argument but 2 were given

In [28]:
standard_arg(arg=1)

1


### 3.2 仅限位置参数

形参的顺序很重要，且这些形参不能用关键字传递。/ 用于在逻辑上分割仅限位置形参与其它形参。如果函数定义中没有 /，则表示没有仅限位置形参。

In [29]:
def pos_only_arg(arg, /):
    print(arg)

In [30]:
pos_only_arg(1)

1


In [31]:
pos_only_arg(arg=1)

TypeError: pos_only_arg() got some positional-only arguments passed as keyword arguments: 'arg'

### 3.3 仅限关键字参数

In [35]:
def kwd_only_arg(*, arg):
    print(arg)

In [37]:
kwd_only_arg(arg=1)

1


In [38]:
kwd_only_arg(1)

TypeError: kwd_only_arg() takes 0 positional arguments but 1 was given

In [39]:
def combined_arg(pos_only, /, standard, *, kwd_only):
    print(pos_only, standard, kwd_only)

In [45]:
combined_arg(1, standard=1, kwd_only=2)

1 1 2


下面的函数定义中，kwds 把 name 当作键，因此，可能与位置参数 name 产生潜在冲突：

In [48]:
def foo(name, **kwds):
    return 'name' in kwds

In [50]:
foo(1, **{'name': 2})

TypeError: foo() got multiple values for argument 'name'

加上 / （仅限位置参数）后，就可以了。此时，函数定义把 name 当作位置参数，'name' 也可以作为关键字参数的键：

In [55]:
def foo(name, /, **kwds):
    return 'name' in kwds

In [56]:
foo(1, **{'name': 2})

True

In [57]:
foo(1, name=2)

True

## 4. 任意实参列表

用于采集传递给函数的所有剩余参数，因此，它们通常在形参列表的末尾。*args 形参后的任何形式参数只能是仅限关键字参数，即只能用作关键字参数，不能用作位置参数：

In [58]:
def contcat(*args, sep="/"):
    return sep.join(args)

In [59]:
contcat("earth", "mars", "venus")

'earth/mars/venus'

## 5. 解包实参列表

函数调用要求独立的位置参数，但实参在列表或元组里时，要执行相反的操作。例如，内置的 range() 函数要求独立的 start 和 stop 实参。如果这些参数不是独立的，则要在调用函数时，用 * 操作符把实参从列表或元组解包出来：

In [60]:
list(range(3, 6))

[3, 4, 5]

In [61]:
# 参数在列表中,无法调用的

args = [3, 6]
list(range(args))

TypeError: 'list' object cannot be interpreted as an integer

In [62]:
# 参数在列表中,进行解包调用
args = [3, 6]
list(range(*args))

[3, 4, 5]

## 6. Lambda表达式

lambda 关键字用于创建小巧的匿名函数。lambda a, b: a+b 函数返回两个参数的和。Lambda 函数可用于任何需要函数对象的地方。在语法上，匿名函数只能是单个表达式。在语义上，它只是常规函数定义的语法糖。与嵌套函数定义一样，lambda 函数可以引用包含作用域中的变量:

In [63]:
def make_incrementor(n):
    return lambda x: x + n

In [66]:
incrementor = make_incrementor(10)

incrementor(1)

11

上面的 incrementor 其实就是:

In [ ]:
def incrementor(x):
    return x + 10

另一种用法是传入一个小函数作为参数。 例如，list.sort() 接受一个排序键函数 key，它可以是一个 lambda 函数:

In [67]:
pairs = [(1, 'one'), (2, 'two'), (3, 'three'), (4, 'four')]
pairs.sort(key=lambda pair: pair[1])
pairs

[(4, 'four'), (1, 'one'), (3, 'three'), (2, 'two')]

## 7. 文档字符串

以下是文档字符串内容和格式的约定。

第一行应为对象用途的简短摘要。为保持简洁，不要在这里显式说明对象名或类型，因为可通过其他方式获取这些信息（除非该名称碰巧是描述函数操作的动词）。这一行应以大写字母开头，以句点结尾。

文档字符串为多行时，第二行应为空白行，在视觉上将摘要与其余描述分开。后面的行可包含若干段落，描述对象的调用约定、副作用等。

Python 解析器不会删除 Python 中多行字符串字面值的缩进，因此，文档处理工具应在必要时删除缩进。这项操作遵循以下约定：文档字符串第一行 之后 的第一个非空行决定了整个文档字符串的缩进量（第一行通常与字符串开头的引号相邻，其缩进在字符串中并不明显，因此，不能用第一行的缩进），然后，删除字符串中所有行开头处与此缩进“等价”的空白符。不能有比此缩进更少的行，但如果出现了缩进更少的行，应删除这些行的所有前导空白符。转化制表符后（通常为 8 个空格），应测试空白符的等效性。

In [ ]:
def f_docstring():
    """摘要部分
    
    其余描述部分
    """
    pass
    

## 8. 函数注解

标注 以字典的形式存放在函数的 __annotations__ 属性中而对函数的其他部分没有影响。 形参标注的定义方式是在形参名后加冒号，后面跟一个会被求值为标注的值的表达式。 返回值标注的定义方式是加组合符号 ->，后面跟一个表达式，这样的校注位于形参列表和表示 def 语句结束的冒号。 下面的示例有一个必须的参数、一个可选的关键字参数以及返回值都带有相应的标注:

In [69]:
def f(ham: str, eggs: str = 'eggs') -> str:
    print("Annotations:", f.__annotations__)
    print("Arguments:", ham, eggs)
    return ham + ' and ' + eggs

In [70]:
f('spam')

Annotations: {'ham': <class 'str'>, 'eggs': <class 'str'>, 'return': <class 'str'>}
Arguments: spam eggs


'spam and eggs'